In [1]:
import pandas as pd
import os
from driver import Driver
import datetime
from collections import Counter

In [2]:
INSTANCE_FOLDER = "/".join([os.path.dirname(os.getcwd()), "createInstance/instances/"])
INSTANCE_FOLDER_CQ = "/".join([os.path.dirname(os.getcwd()), "createInstance/cq/"])
DATA_FOLDER = "/".join([os.path.dirname(os.getcwd()), "createInstance/data/"])

In [3]:
class Instance:
    def __init__(self) -> None:
        self.df_orders = None
        self.df_depots = None
        self.df_tickets = None
        self.df_employee = None
        self.df_depots_loc = None
        self.df_driver = None
        self.list_driver_nbr = []
        self.instance_size = dict()
        self.dict_driver = {}
        self.depot_loc_id = {}
        #self.clean_depot()
        #self.clean_employee()
        #self.clean_tickets()
        #self.get_driver_from_ticket()
        
    
    def init_order(self)->None:
        self.df_orders = pd.read_excel(f"{DATA_FOLDER}/DORDERSTAB_BETON.xlsx", engine='openpyxl', sheet_name="data")
        self.df_orders.sort_values(by='QUANTITY',inplace=True)
        
    def init_depot(self):
        self.df_depots = pd.read_excel(f"{DATA_FOLDER}/adresses_complete.xlsx")
           
    def init_tickets(self):
        self.df_tickets = pd.read_excel(f"{DATA_FOLDER}/DTICKETHISTAB.xlsx")
    
    def init_employee(self):
        self.df_employee = pd.read_excel(f"{DATA_FOLDER}/EMPLOYETAB.xlsx", engine='openpyxl')
        
    def init_depot_loc(self):
         self.df_depots_loc = pd.read_excel(f"{DATA_FOLDER}/LOCATIONTAB.xlsx", engine='openpyxl')
    
    def clean_depot(self) -> None:
        print("Clean Depot")
        self.df_depots = self.df_depots[self.df_depots['Location_type'] != 'customer']
        self.df_depots.dropna(axis=1, inplace=True)
        self.df_depots['NBR'] = self.df_depots['JOB_DESC'].apply(
            lambda x: x if len(x.split(' ')) == 1 else int(x.split(' ')[1]))
        self.df_depots.set_index('NBR', inplace=True)
        
        self.df_depots_loc['LOC_NBR'] = self.df_depots_loc['LOC_NBR'].apply(lambda x: int(x))
       
        self.df_depots_loc.set_index('LOC_NBR', inplace=True)
        

    def clean_employee(self) -> None:
        print("Clean employee")
        self.df_employee.drop(['COMP_NBR', 'emp_sch_specificite'], axis=1, inplace=True)
        self.df_employee.set_index('EMPL_NBR', inplace=True)
        
    def clean_tickets(self) -> None:
        # Les tickets contiennent les séquences des opérations de distribution
        # Un ticket est la livraison de béton sur un chantier par un livreur

        # Je dois déterminer la séquence de livraison par jour.
        self.df_tickets['DATE'] = self.df_tickets['TICKET_DATE'].apply(lambda x: datetime.date(x.year, x.month, x.day))
        self.df_orders['DATE'] = self.df_orders['SHIPDATE'].apply(lambda x: datetime.date(x.year, x.month, x.day))

        self.df_tickets.dropna(axis=0, inplace=True)
        # # Enlever TRUCK_NBR = ACHATEXT
        # ! self.df_tickets = self.df_tickets[self.df_tickets['TRUCK_NBR'] != 'ACHATEXT']
        # # Enlever livraison dont cap > 12
        self.df_tickets = self.df_tickets[self.df_tickets['QUANTITY'] <= 12]
        # # SHIP_LOC = 099
        self.df_tickets = self.df_tickets[self.df_tickets['SHIP_LOC'] != 99]
        
        self.df_tickets['CUST'] = self.df_tickets['ORDER_ID'].apply(lambda x: self.df_orders[self.df_orders['ORDER_ID']==x]['CUST_NBR'].max())
        self.df_tickets['JOB_DESC'] = self.df_tickets['ORDER_ID'].apply(lambda x: self.df_orders[self.df_orders['ORDER_ID']==x]['JOB_DESC'].max())
                              

        # ! self.df_tickets = self.df_tickets[self.df_tickets['DRIVER_NBR'] != 1]
        # ! self.df_tickets = self.df_tickets[self.df_tickets['DRIVER_NBR'] != 999]
        
    def get_driver_from_ticket(self) -> None:
        # Liste des livreurs disponibles dans le fichier Ticket.
        self.df_driver = self.df_tickets.loc[:, ("DTICKETHIS_TICKET_ID", "SHIP_LOC", "ORDER_ID", "DRIVER_NBR",
                                                 "TRUCK_NBR", "SCHED_LOC", "LOAD", "LOADNR", "QUANTITY")]
        # Enlever les ordres où le numero du chauffeur/camion est manquant
        self.df_driver.dropna(axis=0, inplace=True)
        # Enlever TRUCK_NBR = ACHATEXT
        # ! self.df_driver = self.df_driver[self.df_driver['TRUCK_NBR'] != 'ACHATEXT']
        # Enlever livraison dont cap > 12
        self.df_driver = self.df_driver[self.df_driver['QUANTITY'] <= 12]

        df_copy = self.df_driver[['TRUCK_NBR', 'DRIVER_NBR']].copy()
        df_copy.reset_index(inplace=True)
        df_copy.set_index(['TRUCK_NBR', 'DRIVER_NBR'], inplace=True)
        self.list_driver_nbr = list(dict.fromkeys(df_copy.index))
        employee_nbr = self.df_employee.index
        dummy_driver = []
        for (t_nbr, d_nbr) in self.list_driver_nbr:
            if d_nbr not in employee_nbr:
                dummy_driver.append(d_nbr)
            d = Driver()
            d.t_nbr = t_nbr
            d.d_nbr = int(d_nbr)
            _df = self.df_driver[self.df_driver['TRUCK_NBR'] == t_nbr]
            _cap = max(_df['QUANTITY'])
            if _cap <= 8:
                d.capacity = 8
            else:
                d.capacity = 12
            d.factory_nbr = Counter(_df['SHIP_LOC']).most_common(1)[0][0]
            self.dict_driver[d.t_nbr] = d
            
    def get_driver_from_day(self, day):
        # Liste des livreurs disponibles dans le fichier Ticket.
        df_driver = self.df_tickets.loc[:, ("DTICKETHIS_TICKET_ID", "SHIP_LOC", "ORDER_ID", "DRIVER_NBR",
                                                 "TRUCK_NBR", "SCHED_LOC", "LOAD", "LOADNR", "QUANTITY","DATE")]
        # Enlever les ordres où le numero du chauffeur/camion est manquant
        df_driver.dropna(axis=0, inplace=True)
        # Enlever TRUCK_NBR = ACHATEXT
        # ! self.df_driver = self.df_driver[self.df_driver['TRUCK_NBR'] != 'ACHATEXT']
        # Enlever livraison dont cap > 12
        df_driver = df_driver[df_driver['DATE']==day]
        df_driver = df_driver[df_driver['QUANTITY'] <= 12]

        df_copy = df_driver[['TRUCK_NBR', 'DRIVER_NBR']].copy()
        df_copy.reset_index(inplace=True)
        df_copy.set_index(['TRUCK_NBR', 'DRIVER_NBR'], inplace=True)
        list_driver_nbr = list(dict.fromkeys(df_copy.index))
        employee_nbr = self.df_employee.index
        dummy_driver = []
        dict_driver = {}
        for (t_nbr, d_nbr) in list_driver_nbr:
            if d_nbr not in employee_nbr:
                dummy_driver.append(d_nbr)
            d = Driver()
            d.t_nbr = t_nbr
            d.d_nbr = int(d_nbr)
            _df = df_driver[df_driver['TRUCK_NBR'] == t_nbr]
            _cap = max(_df['QUANTITY'])
            if _cap <= 8:
                d.capacity = 8
            else:
                d.capacity = 12
            d.factory_nbr = Counter(_df['SHIP_LOC']).most_common(1)[0][0]
            dict_driver[d.t_nbr] = d
            
        return dict_driver
            
    def show(self):
        print(f" {len(self.df_orders)} ordres")
        print(f" {len(self.df_driver)} chauffeurs")
        print(f" {len(self.df_depots)} depots")

        
    def create_instance(self, jour) -> None:
        day_op = self.df_tickets[self.df_tickets['DATE'] == jour]
        day_op = day_op[
            ["DTICKETHIS_TICKET_ID", "SHIP_LOC", "ORDER_ID", "DRIVER_NBR", "TRUCK_NBR", "SCHED_LOC", "LOAD", "LOADNR",
             "QUANTITY", "BEGIN_LOAD", "FINISH_LOAD", "ARRIVE_PLANT"]]
        day = jour.day
        if day < 10:
            day = f"0{day}"
        print(f"Instance {day}")
        with open(f"{INSTANCE_FOLDER}/{jour.year}{jour.month}{day}.txt", 'w') as file:
            contents = []
            order_count = 0
            dmd_total = 0
            contents.append(f"Depots:   {len(self.df_depots):8}\n")
            orders_of_day = list(dict.fromkeys(day_op['ORDER_ID']))
            all_orders_of_day = list(dict.fromkeys(self.df_orders['ORDER_ID'][self.df_orders['DATE'] == jour]))
            real_orders_of_day = list(set(all_orders_of_day)&set(orders_of_day))
            print(f"{len(real_orders_of_day)} ordres servis")

            # 2 Nombre de clients
            contents.append(f"Clients:   {len(real_orders_of_day):7}\n")

            list_driver = list(dict.fromkeys(day_op['TRUCK_NBR']))
            print(jour)

            contents.append(f"Drivers:  {len(list_driver):8}\n")

            # 4 Liste des depots
            # for i,d in enumerate(list_depot):
            #     contents.append(f"{i} {df_depots.loc[d]['Location ID']} {d} {df_depots_loc.loc[d]['CAPACITY']}\n")  
            for i, dep in enumerate(self.df_depots.index):
                contents.append(
                    f"{i:2} {self.df_depots.loc[dep]['Location ID']} {dep:2} {int(self.df_depots_loc.loc[dep]['CAPACITY']):3} \n")
                self.depot_loc_id[dep] = i
            # 5 Liste des clients de la journée  
            for i, ord in enumerate(real_orders_of_day):
                ordres = self.df_orders[self.df_orders['ORDER_ID'] == ord]
                if len(ordres) == 0:
                    continue

                ordre = ordres.iloc[0]
                string = f"{i:3} {ord} {ordre['CUST_NBR']} {ordre['LOCATION ID']:4} {self.depot_loc_id[ordre['SCHED_LOC']]:2} {ordre['SHIPTIME (min)']:4.0f} {ordres['QUANTITY'].sum():5.1f} {ordres['QUANTITY'].count():2} \n"
                contents.append(string)
                dmd_total = dmd_total + ordres['QUANTITY'].sum()
            # 6 Liste des ordres de la journée  
            for clientID, ord in enumerate(real_orders_of_day):

                ord_seq = day_op[day_op['ORDER_ID'] == ord]
                ordres = self.df_orders[self.df_orders['ORDER_ID'] == ord]
                if len(ordres) == 0:
                    continue
                for j in range(len(ordres)):
                    ordre = ordres.iloc[j]
                    contents.append(
                        f"{order_count:3} {clientID:3} {ordre['QUANTITY']:5.1f} \n")
                    order_count = order_count + 1

            # 7 Ajouter les chauffeurs
            # Lines of drivers: driver id, driver number, driver rank, depot assigned driver max capacity
            driver_cap = set()
            dict_driver = self.get_driver_from_day(jour)

            for i, id in enumerate(list_driver):
                
                if id not in dict_driver.keys():
                    continue
                
                dr = dict_driver[id]
                df_drv = day_op[day_op['TRUCK_NBR'] == dr.t_nbr]
                _data = df_drv[["BEGIN_LOAD", "ARRIVE_PLANT"]]
                _data.sort_values(["BEGIN_LOAD"], inplace=True)
                # print(dr)
                # _data = test[]
                begin, end = _data.iloc[0, 0], _data.iloc[-1, 1]
                factory_nbr = Counter(df_drv['SHIP_LOC']).most_common(1)[0][0]
                contents.append(
                    f"{i:3} {dr.d_nbr:8} {'_'.join(dr.t_nbr.split(' ')):8} {dr.rank:6} {self.depot_loc_id[factory_nbr]:2} {dr.capacity:2.0f}"
                    f" {int((begin.hour * 60 + begin.minute) / 60) * 60:4}\n")
                driver_cap.add(dr.capacity)
               
                
            contents.insert(2, f"Orders:  {order_count:9}  \n")
            contents[-1] = contents[-1].rstrip()
            contents = "".join(contents)
            file.write(contents)
            self.instance_size[f"{jour.year}{jour.month}{day}"] = [len(self.df_depots), len(real_orders_of_day),
                                                                   order_count,
                                                                   len(list_driver), dmd_total, len(driver_cap)]


   

        pd.DataFrame.from_dict(self.instance_size, orient='index').to_csv(f'{INSTANCE_FOLDER}/instance_size.csv',   sep=';')


In [50]:
def create_instance_CQ(Object, jour) -> None:
    
        day_op = Object.df_tickets[Object.df_tickets['DATE'] == jour]

        day_op = day_op[
            ["DTICKETHIS_TICKET_ID", "SHIP_LOC", "ORDER_ID", "DRIVER_NBR", "TRUCK_NBR", "SCHED_LOC", "LOAD", "LOADNR",
             "QUANTITY", "BEGIN_LOAD", "FINISH_LOAD", "ARRIVE_PLANT","PROD_NBR","CUST","JOB_DESC"]]
           
        day = jour.day
        if day < 10:
            day = f"0{day}"
        print(f"Instance {day}")
        with open(f"{INSTANCE_FOLDER_CQ}/{jour.year}{jour.month}{day}.txt", 'w') as file:
            contents = []
            
            print(jour, ' ')
            chauffeurs = list(dict.fromkeys(day_op['TRUCK_NBR']))
            list_driver = set()
            for driver in chauffeurs:
                driver_orders = list(dict.fromkeys(newInstance.df_tickets['ORDER_ID'][newInstance.df_tickets['TRUCK_NBR']==driver]))
                for ord in driver_orders:
                    job = list(dict.fromkeys(newInstance.df_orders['JOB_DESC'][newInstance.df_orders['ORDER_ID']==ord]))
   
                    if 'PICKUP' not in job:
                        list_driver.add(driver)
                        break
                    else:
                        if len(job)>0:
                            list_driver.add(driver)
                            
            list_driver = list(list_driver)
            print(len(chauffeurs),' VS ',len(list_driver))
            
            dict_driver = Object.get_driver_from_day(jour)
            
            usedLoc = set()
            for i, id in enumerate(dict_driver):
                if id not in list_driver:
                    continue

                dr = dict_driver[id]
                df_drv = day_op[day_op['TRUCK_NBR'] == dr.t_nbr]
                _data = df_drv[["BEGIN_LOAD", "ARRIVE_PLANT"]].copy()
                _data.sort_values(["BEGIN_LOAD"], inplace=True)
                usedLoc.add(Counter(df_drv['SHIP_LOC']).most_common(1)[0][0])
               
            count = 0

            for dep in usedLoc:    
                if dep not in Object.df_depots.index:
                    continue
                Object.depot_loc_id[dep] = count
                count = count + 1
        
            #  Ajouter les chauffeurs
            # Lines of drivers: driver id, driver number, driver rank, depot assigned driver max capacity
            driver_cap = set()
            dict_driver = Object.get_driver_from_day(jour)
            dict_driver = dict(sorted(dict_driver.items(), key=lambda x:x[1].capacity))
            
            contents.append(f"Drivers:  {len(list_driver):8}\n")
            
            for i, id in enumerate(dict_driver):
                
                if id not in list_driver:
                    continue
                
                dr = dict_driver[id]
                df_drv = day_op[day_op['TRUCK_NBR'] == dr.t_nbr]
                _data = df_drv[["BEGIN_LOAD", "ARRIVE_PLANT"]].copy()
                _data.sort_values(["BEGIN_LOAD"], inplace=True)
                begin, end = _data.iloc[0, 0], _data.iloc[-1, 1]
                factory_nbr = Counter(df_drv['SHIP_LOC']).most_common(1)[0][0]
                contents.append(
                    f"{i:3} {dr.d_nbr:8} {'_'.join(dr.t_nbr.split(' ')):8} {dr.rank:6} {Object.depot_loc_id[factory_nbr]:2} {dr.capacity:2.0f}"
                    f" {int((begin.hour * 60 + begin.minute) / 60) * 60:4}\n")
                driver_cap.add(dr.capacity)
            
            
            orders_of_day = list(dict.fromkeys(day_op['ORDER_ID']))
            all_orders_of_day = list(dict.fromkeys(Object.df_orders['ORDER_ID'][Object.df_orders['DATE'] == jour]))
            real_orders_of_day = list(set(orders_of_day))
            print(f"{len(orders_of_day)} {len(all_orders_of_day)} {len(real_orders_of_day)}")
            all_cust_of_day = list(dict.fromkeys(day_op['CUST']))
            
            new_orders = list()

            Object.df_orders.sort_values(['QUANTITY'], inplace=True)
            Map_Qt = dict()
            
            for i, cust in enumerate(all_cust_of_day):
                cust_ordres = list(dict.fromkeys(day_op['ORDER_ID'][day_op['CUST'] == cust][day_op['JOB_DESC'] != 'PICKUP']))
                _sum = 0
#                 print('cust ',cust,' ',cust_ordres)
                for ord in cust_ordres:
                    products = list(dict.fromkeys(Object.df_orders['PROD_NBR'][Object.df_orders['ORDER_ID'] == ord][Object.df_orders['JOB_DESC'] != 'PICKUP']))
                    if len(products) == 0:
                        continue

                    for prod in products:
                    
                        prod_day_op = day_op[day_op['PROD_NBR']==prod][day_op['ORDER_ID']==ord]
                        demand = prod_day_op['QUANTITY'].sum()
                        if demand < 1:
                            continue
#                         print('ord ' ,ord,' ',prod,' ',demand)
                        
                        _sum = _sum + demand
                if _sum < 1:
                    continue
                Map_Qt[cust] = _sum
                
            sortedMap = sorted(Map_Qt.items(), key=lambda x:x[1])
            clientCount = 0
            dmd_total = 0.0
            for i, cust in enumerate(sortedMap):
                
                cust_ordres = list(dict.fromkeys(day_op['ORDER_ID'][day_op['CUST'] == cust[0]][day_op['JOB_DESC'] != 'PICKUP']))
#                 print('cust ',cust[0],':',cust[1])
                
                products = list(dict.fromkeys(day_op['PROD_NBR'][day_op['CUST'] == cust[0]][day_op['JOB_DESC'] != 'PICKUP']))
                
                if len(products) == 0 :
                    continue
                
                if not any(item in orders_of_day for item in cust_ordres):
                    continue 
                    
                for ord in cust_ordres:
                    ordres = Object.df_orders[Object.df_orders['ORDER_ID'] == ord]
                    prod_nbrs = list(dict.fromkeys(ordres['PROD_NBR']))
                    ticket_ordres = day_op[day_op['ORDER_ID'] == ord]
                    ticket_ordres = ticket_ordres[ticket_ordres['PROD_NBR'].isin(prod_nbrs)]
                    if ticket_ordres['QUANTITY'].sum() < 1:
                        continue
                    dmd_total = dmd_total + ticket_ordres['QUANTITY'].sum()
                    ordre = ordres.iloc[0]
                    
                    cur_order_count = 0
                    for val in prod_nbrs:
                        
                        if ticket_ordres[ticket_ordres['PROD_NBR'] == val]['QUANTITY'].sum() > 0:
                            cur_order_count = cur_order_count + 1
                    
                    string = f"{clientCount:3} {ord} {cust[0]} {ordre['LOCATION ID']:4} {Object.depot_loc_id[ordre['SCHED_LOC']]:2} {ordre['SHIPTIME (min)']:4.0f} {ticket_ordres['QUANTITY'].sum():5.1f} {cur_order_count:2} \n"
                    contents.append(string)
                    new_orders.append(ord)
                    clientCount = clientCount + 1
            
            contents.insert(-clientCount, f"Clients:  {clientCount} {dmd_total}\n")
            
            print('Client Demand ',dmd_total)
                    
            # 6 Liste des ordres de la journée  

            order_count = 0    
            sum_order_dmd = 0
            for clientID, ord in enumerate(new_orders):

                ord_seq = day_op[day_op['ORDER_ID'] == ord]
                ordres = Object.df_orders[Object.df_orders['ORDER_ID'] == ord]
                prod_nbrs = list(dict.fromkeys(ordres['PROD_NBR']))
                
                if len(ordres) == 0:
                    continue
                    
                for prod in prod_nbrs:
                    ticket_ordres = day_op[day_op['ORDER_ID'] == ord]
                    
                    ticket_ordres =  ord_seq[ord_seq['PROD_NBR']==prod]
                    if ticket_ordres['QUANTITY'].sum() < 1:
                        continue
                    sum_order_dmd = sum_order_dmd + ticket_ordres['QUANTITY'].sum()
                    
                    contents.append(f"{order_count:3} {clientID:3} {ticket_ordres['QUANTITY'].sum():5.1f} \n")
                    order_count = order_count + 1
                    
            contents.insert(-order_count, f"Orders:  {order_count}  \n")
            
            print('order_demand ',sum_order_dmd)
            
            contents.append(f"Plants:   {len(usedLoc)}\n")
           
            list_driver = list(dict.fromkeys(day_op['TRUCK_NBR']))

            # 4 Liste des depots
             
            for i, dep in enumerate(usedLoc):
                contents.append(
                    f"{i:2} {Object.df_depots.loc[dep]['Location ID']} {dep:2} {int(Object.df_depots_loc.loc[dep]['CAPACITY']):3} \n")
#                 Object.depot_loc_id[dep] = i
            
            # 5 Liste des clients de la journée  
            # 2 Nombre de clients
                
#             contents.insert(2, f"Orders:  {order_count:9}  \n")
            contents[-1] = contents[-1].rstrip()
            contents = "".join(contents)
            file.write(contents)
            Object.instance_size[f"{jour.year}{jour.month}{day}"] = [len(usedLoc), clientCount,
                                                                   order_count,
                                                                   len(list_driver), dmd_total, len(driver_cap)]
        pd.DataFrame.from_dict(Object.instance_size, orient='index').to_csv(f'{INSTANCE_FOLDER_CQ}/instance_size.csv',
                                                                            sep=';',decimal='.')

create_instance_CQ(newInstance,datetime.date(2020, 10, 2))

Instance 02
2020-10-02  
133  VS  133
80 98 80
Client Demand  2951.5
order_demand  2951.5


/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_60211/1138092531.py:102: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prod_day_op = day_op[day_op['PROD_NBR']==prod][day_op['ORDER_ID']==ord]


In [49]:
def create_instance_article(Object, jour) -> None:
    
        day_op = Object.df_tickets[Object.df_tickets['DATE'] == jour]

        day_op = day_op[
            ["DTICKETHIS_TICKET_ID", "SHIP_LOC", "ORDER_ID", "DRIVER_NBR", "TRUCK_NBR", "SCHED_LOC", "LOAD", "LOADNR",
             "QUANTITY", "BEGIN_LOAD", "FINISH_LOAD", "ARRIVE_PLANT","PROD_NBR","CUST","JOB_DESC"]]
           
        day = jour.day
        if day < 10:
            day = f"0{day}"
        print(f"Instance {day}")
        with open(f"{INSTANCE_FOLDER}/{jour.year}{jour.month}{day}.rmc", 'w') as file:
            contents = []
            
            print(jour, ' ')
            chauffeurs = list(dict.fromkeys(day_op['TRUCK_NBR']))
            list_driver = set()
            for driver in chauffeurs:
                driver_orders = list(dict.fromkeys(newInstance.df_tickets['ORDER_ID'][newInstance.df_tickets['TRUCK_NBR']==driver]))
                for ord in driver_orders:
                    job = list(dict.fromkeys(newInstance.df_orders['JOB_DESC'][newInstance.df_orders['ORDER_ID']==ord]))
   
                    if 'PICKUP' not in job:
                        list_driver.add(driver)
                        break
                    else:
                        if len(job)>0:
                            list_driver.add(driver)
                            
            list_driver = list(list_driver)
            print(len(chauffeurs),' VS ',len(list_driver))
            
            dict_driver = Object.get_driver_from_day(jour)
            
            usedLoc = set()
            for i, id in enumerate(dict_driver):
                if id not in list_driver:
                    continue

                dr = dict_driver[id]
                df_drv = day_op[day_op['TRUCK_NBR'] == dr.t_nbr]
                _data = df_drv[["BEGIN_LOAD", "ARRIVE_PLANT"]].copy()
                _data.sort_values(["BEGIN_LOAD"], inplace=True)
                usedLoc.add(Counter(df_drv['SHIP_LOC']).most_common(1)[0][0])
               
            count = 0

            for dep in usedLoc:    
                if dep not in Object.df_depots.index:
                    continue
                Object.depot_loc_id[dep] = count
                count = count + 1
        
            #  Ajouter les chauffeurs
            # Lines of drivers: driver id, driver number, driver rank, depot assigned driver max capacity
            driver_cap = set()
            dict_driver = Object.get_driver_from_day(jour)
            dict_driver = dict(sorted(dict_driver.items(), key=lambda x:x[1].capacity))
            
            contents.append(f"Drivers:  {len(list_driver):8}\n")
            
            for i, id in enumerate(dict_driver):
                
                if id not in list_driver:
                    continue
                
                dr = dict_driver[id]
                df_drv = day_op[day_op['TRUCK_NBR'] == dr.t_nbr]
                _data = df_drv[["BEGIN_LOAD", "ARRIVE_PLANT"]].copy()
                _data.sort_values(["BEGIN_LOAD"], inplace=True)
                begin, end = _data.iloc[0, 0], _data.iloc[-1, 1]
                factory_nbr = Counter(df_drv['SHIP_LOC']).most_common(1)[0][0]
                contents.append(
                    f"{i:3} {Object.depot_loc_id[factory_nbr]:2} {dr.capacity:2.0f}"
                    f" {int((begin.hour * 60 + begin.minute) / 60) * 60:4}\n")

                driver_cap.add(dr.capacity)
            
            
            orders_of_day = list(dict.fromkeys(day_op['ORDER_ID']))
            all_orders_of_day = list(dict.fromkeys(Object.df_orders['ORDER_ID'][Object.df_orders['DATE'] == jour]))
            real_orders_of_day = list(set(orders_of_day))
            print(f"{len(orders_of_day)} {len(all_orders_of_day)} {len(real_orders_of_day)}")
            all_cust_of_day = list(dict.fromkeys(day_op['CUST']))
            
            new_orders = list()

            Object.df_orders.sort_values(['QUANTITY'], inplace=True)
            Map_Qt = dict()
            
            for i, cust in enumerate(all_cust_of_day):
                cust_ordres = list(dict.fromkeys(day_op['ORDER_ID'][day_op['CUST'] == cust][day_op['JOB_DESC'] != 'PICKUP']))
                _sum = 0
#                 print('cust ',cust,' ',cust_ordres)
                for ord in cust_ordres:
                    products = list(dict.fromkeys(Object.df_orders['PROD_NBR'][Object.df_orders['ORDER_ID'] == ord][Object.df_orders['JOB_DESC'] != 'PICKUP']))
                    if len(products) == 0:
                        continue

                    for prod in products:
                    
                        prod_day_op = day_op[day_op['PROD_NBR']==prod][day_op['ORDER_ID']==ord]
                        demand = prod_day_op['QUANTITY'].sum()
                        if demand < 1:
                            continue
                        
                        _sum = _sum + demand
                if _sum < 1:
                    continue
                Map_Qt[cust] = _sum
                
            sortedMap = sorted(Map_Qt.items(), key=lambda x:x[1])
            clientCount = 0
            dmd_total = 0.0
            for i, cust in enumerate(sortedMap):
                
                cust_ordres = list(dict.fromkeys(day_op['ORDER_ID'][day_op['CUST'] == cust[0]][day_op['JOB_DESC'] != 'PICKUP']))
                
                products = list(dict.fromkeys(day_op['PROD_NBR'][day_op['CUST'] == cust[0]][day_op['JOB_DESC'] != 'PICKUP']))
                
                if len(products) == 0 :
                    continue
                
                if not any(item in orders_of_day for item in cust_ordres):
                    continue 
                    
                for ord in cust_ordres:
                    ordres = Object.df_orders[Object.df_orders['ORDER_ID'] == ord]
                    prod_nbrs = list(dict.fromkeys(ordres['PROD_NBR']))
                    ticket_ordres = day_op[day_op['ORDER_ID'] == ord]
                    ticket_ordres = ticket_ordres[ticket_ordres['PROD_NBR'].isin(prod_nbrs)]
                    cur_order_count = 0
                    for val in prod_nbrs:
                        
                        if ticket_ordres[ticket_ordres['PROD_NBR'] == val]['QUANTITY'].sum() > 0:
                            cur_order_count = cur_order_count + 1
                    
                    dmd_total = dmd_total + ticket_ordres['QUANTITY'].sum()
                    ordre = ordres.iloc[0]
                    string = f"{clientCount:3} {ordre['LOCATION ID']:4} {Object.depot_loc_id[ordre['SCHED_LOC']]:2} {ordre['SHIPTIME (min)']:4.0f} {ordres['QUANTITY'].sum():5.1f} {cur_order_count:2} \n"
                    
#                     string = f"{clientCount:3} {ord} {cust[0]} {ordre['LOCATION ID']:4} {Object.depot_loc_id[ordre['SCHED_LOC']]:2} {ordre['SHIPTIME (min)']:4.0f} {ticket_ordres['QUANTITY'].sum():5.1f} {ordres['QUANTITY'].count():2} \n"
                    contents.append(string)
                    new_orders.append(ord)
                    clientCount = clientCount + 1
            
            contents.insert(-clientCount, f"Clients:  {clientCount} {dmd_total}\n")
            
            print('Client Demand ',dmd_total)
                    
            # 6 Liste des ordres de la journée  

            order_count = 0    
            sum_order_dmd = 0
            for clientID, ord in enumerate(new_orders):

                ord_seq = day_op[day_op['ORDER_ID'] == ord]
                ordres = Object.df_orders[Object.df_orders['ORDER_ID'] == ord]
                prod_nbrs = list(dict.fromkeys(ordres['PROD_NBR']))
                
                if len(ordres) == 0:
                    continue
                    
                for prod in prod_nbrs:
                    ticket_ordres = day_op[day_op['ORDER_ID'] == ord]
                    
                    ticket_ordres =  ord_seq[ord_seq['PROD_NBR']==prod]
                    if ticket_ordres['QUANTITY'].sum() < 1:
                        continue
                    sum_order_dmd = sum_order_dmd + ticket_ordres['QUANTITY'].sum()
                   
                    contents.append(f"{order_count:3} {clientID:3} {ticket_ordres['QUANTITY'].sum():5.1f} \n")
                    order_count = order_count + 1
                    
            contents.insert(-order_count, f"Orders:  {order_count}  \n")
            
            print('order_demand ',sum_order_dmd)
            
            contents.append(f"Plants:   {len(usedLoc)}\n")
           
            list_driver = list(dict.fromkeys(day_op['TRUCK_NBR']))

            # 4 Liste des depots
             
            for i, dep in enumerate(usedLoc):
                contents.append(
                    f"{i:2} {Object.df_depots.loc[dep]['Location ID']} {int(Object.df_depots_loc.loc[dep]['CAPACITY']):3} \n")
            
            # 5 Liste des clients de la journée  
            # 2 Nombre de clients
                
#             contents.insert(2, f"Orders:  {order_count:9}  \n")
            contents[-1] = contents[-1].rstrip()
            contents = "".join(contents)
            file.write(contents)
            Object.instance_size[f"{jour.year}{jour.month}{day}"] = [len(usedLoc), clientCount,
                                                                   order_count,
                                                                   len(list_driver), dmd_total, len(driver_cap)]
        pd.DataFrame.from_dict(Object.instance_size, orient='index').to_csv(f'{INSTANCE_FOLDER}/instance_size.csv',
                                                                            sep=';',decimal='.')

create_instance_article(newInstance,datetime.date(2020, 10, 1))

Instance 01
2020-10-01  
136  VS  136
104 128 104


/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_60211/2330858190.py:103: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prod_day_op = day_op[day_op['PROD_NBR']==prod][day_op['ORDER_ID']==ord]


Client Demand  3078.0
order_demand  3078.0


In [51]:
 def create_all_instances(Object) -> None:
        print("Create all instances")
        Object.instance_size=dict()
        Object.instance_size[f"date"]=['depot','client','order','driver','demand','cap']
        ticket_date = list(dict.fromkeys(Object.df_tickets['DATE']))
#         ticket_date = [datetime.date(2020, 10, 10)]
        for jour in ticket_date:
            create_instance_CQ(Object,jour)
#             create_instance_article(Object,jour)            

In [52]:
create_all_instances(newInstance)

Create all instances
Instance 01
2020-10-01  
136  VS  136
104 128 104


/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_60211/1138092531.py:102: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prod_day_op = day_op[day_op['PROD_NBR']==prod][day_op['ORDER_ID']==ord]


Client Demand  3078.0
order_demand  3078.0
Instance 02
2020-10-02  
133  VS  133
80 98 80


/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_60211/1138092531.py:102: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prod_day_op = day_op[day_op['PROD_NBR']==prod][day_op['ORDER_ID']==ord]


Client Demand  2951.5
order_demand  2951.5
Instance 03
2020-10-03  
35  VS  35
11 17 11
Client Demand  651.5
order_demand  651.5
Instance 05
2020-10-05  


...

/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_60211/1138092531.py:102: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prod_day_op = day_op[day_op['PROD_NBR']==prod][day_op['ORDER_ID']==ord]


Client Demand  3129.5
order_demand  3129.5
Instance 28
2020-11-28  
31  VS  31
8 9 8
Client Demand  776.0
order_demand  776.0
Instance 30
2020-11-30  


/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_60211/1138092531.py:102: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prod_day_op = day_op[day_op['PROD_NBR']==prod][day_op['ORDER_ID']==ord]


118  VS  118
62 90 62
Client Demand  2327.5
order_demand  2327.5


/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_60211/1138092531.py:102: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prod_day_op = day_op[day_op['PROD_NBR']==prod][day_op['ORDER_ID']==ord]


In [124]:
chauffeurs = list(set(newInstance.df_tickets['TRUCK_NBR']))
pickup=list()
for driver in chauffeurs:
    driver_orders = list(dict.fromkeys(newInstance.df_tickets['ORDER_ID'][newInstance.df_tickets['TRUCK_NBR']==driver]))
    for ord in driver_orders:
        job = list(dict.fromkeys(newInstance.df_orders['JOB_DESC'][newInstance.df_orders['ORDER_ID']==ord]))
#         tasks = newInstance.df_orders[['JOB_DESC','SHIP_DATE']][newInstance.df_orders['ORDER_ID']==ord]
        
        if 'PICKUP' in job:
            pickup.append(driver)
#             day = list(dict.fromkeys(tasks['SHIP_DATE'][newInstance.df_orders['ORDER_ID']==ord]))
            print(driver,' ',ord, ' ',len(driver_orders))
            break
#     print(driver_orders)
#     break

    

489   924806   95
155   924806   67
B999   928268   5
P999   927010   12
470   933136   100
LV999   928002   6
CH999   930835   7


In [8]:
newInstance = Instance()

In [9]:
newInstance.init_order()

In [10]:
newInstance.init_depot()

In [11]:
newInstance.init_tickets()

In [12]:
newInstance.init_employee()

In [13]:
newInstance.init_depot_loc()

In [14]:
newInstance.clean_depot()   

Clean Depot


In [15]:
newInstance.clean_employee()

Clean employee


In [16]:
newInstance.clean_tickets()

In [70]:
create_instance(newInstance, datetime.date(2020, 10, 31))

Instance 31
4 ordres servis
2020-10-31


/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_93690/1765312482.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"], inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_93690/1765312482.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"], inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_93690/1765312482.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

NameError: name 'self' is not defined

In [56]:
newInstance.df_orders

,COMP_NBR,ORDER_ID,ORDER_LOC,ORDER_NBR,ORDER_STATUS,ORDER_DATE,ORDER_TIME (min),SHIPDATE,Day_of_the_week,SHIPTIME (min),...,UNIT_OF_MEASURE,QUANTITY,LOOKUP_DESC,CONFIG_DESC,LOCATION ID,Unnamed: 33,Unnamed: 34,Location,ID,DATE
4215,1,932728,0,618637,2,2020-10-30,588.0,2020-11-03,Tue,690.0,...,M3,0.5,BASE,BEGIN REDI MIX,202,NaN,NaN,NaN,NaN,2020-11-03
1253,1,926332,0,612468,2,2020-10-01,654.0,2020-11-03,Tue,840.0,...,M3,0.5,REMBLAI SANS RETRAIT,REMBLAI,462,NaN,NaN,NaN,NaN,2020-11-03
4244,1,932777,0,618686,2,2020-10-30,600.0,2020-11-11,Wed,900.0,...,M3,0.5,BASE,BEGIN REDI MIX,202,NaN,NaN,NaN,NaN,2020-11-11
4245,1,932780,0,618689,2,2020-10-30,602.0,2020-11-13,Fri,870.0,...,M3,0.5,BASE,BEGIN REDI MIX,202,NaN,NaN,NaN,NaN,2020-11-13
4246,1,932781,0,618690,2,2020-10-30,602.0,2020-11-16,Mon,870.0,...,M3,0.5,BASE,BEGIN REDI MIX,202,NaN,NaN,NaN,NaN,2020-11-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,1,926685,57,612811,0,2020-10-02,820.0,2020-10-23,Fri,415.0,...,M3,376.0,DALLE SUR SOL,25 MPA,533,NaN,NaN,NaN,NaN,2020-10-23
1289,1,926379,5,612515,2,2020-10-01,719.0,2020-10-07,Wed,420.0,...,M3,400.0,DALLE STRUCTURALE,35 MPA,57,NaN,NaN,NaN,NaN,2020-10-07
1162,1,926133,58,612275,0,2020-09-30,789.0,2020-10-17,Sat,390.0,...,M3,522.0,DALLE STRUCTURALE,25 MPA,423,NaN,NaN,NaN,NaN,2020-10-17
161,1,922567,5,608801,2,2020-09-15,630.0,2020-11-27,Fri,420.0,...,M3,535.0,DALLE SUR SOL,35 MPA,9,NaN,NaN,NaN,NaN,2020-11-27


In [380]:
newInstance.create_all_instances()

Create all instances
Instance 01
103 ordres servis
2020-10-01
Instance 02
77 ordres servis
2020-10-02


/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"], inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"], inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

Instance 03
10 ordres servis
2020-10-03
Instance 05
85 ordres servis
2020-10-05
Instance 06
115 ordres servis
2020-10-06


/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"], inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"], inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

Instance 07
66 ordres servis
2020-10-07
Instance 08
101 ordres servis
2020-10-08


...

/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"], inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"], inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

Instance 25
81 ordres servis
2020-11-25
Instance 26
84 ordres servis
2020-11-26


/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"], inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"], inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

Instance 27
79 ordres servis
2020-11-27
Instance 28
8 ordres servis
2020-11-28
Instance 30
62 ordres servis
2020-11-30


/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"], inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"], inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_79994/2234718423.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/